In [3]:
import requests
from datetime import datetime
import smtplib
import time
import pandas as pd
import xlrd
import boto3
import os
from twilio.rest import Client

In [4]:
apiKey = '...' # API key obtained from the DVSA

In [5]:
# We need an excel or csv file that contains vehicle data that we need to search against the MOT database.
# Can also use any other file format as long as it has tabular data that we can iterate over.

df = pd.read_excel(r'vehicles.xlsx')
registrations = df['registration'].tolist()  # Converting the registrations series to a list
mobile = df['mobile'].tolist()  # Converting the mobile numbers series to a list
email = df['email'].tolist()  # Converting the email series to a list

In [6]:
# Function to extract data from MOT database and return a list of days left for all the vehicles from our df

def check_days_left(p):
    headers = {
        'Accept': 'application/json',
        'x-api-key': apiKey,
    }
    params = (
        ('registration', p),
    )
    response = requests.get('https://beta.check-mot.service.gov.uk/trade/vehicles/mot-tests',
                            headers=headers, params=params)
    r = response.json()  # This returns a list of dicts of the full MOT History data
    carDetails = r[0]  # Access the first dictionary in the list. This contains the MOT data for the vehicles specified.
    motTests = carDetails['motTests']  # This returns a list of dicts of each MOT test corresponding to the vehicles specified.
    latestTest = motTests[0]  # Access the first dictionary to get the details of the latest test which will be used to find out when it expires.
    expDate = latestTest['expiryDate']  # This returns the expiry date key of the latest test dictionary.
    dtformat = datetime.strptime(expDate, '%Y.%m.%d')  # Convert expDate to datetime format.
    difference = dtformat - datetime.now()  # Gets the timedelta between now and expiry date.
    days_left = difference.days  # Extract the number of days from the above as an int.
    return days_left

In [7]:
# Function to call the check_days_left() function on our registrations list in order to get the days left until expiry of MOT for all the vehicles in the list.

def get_days_left():
    list_days_left = []
    for reg in registrations:
        list_days_left.append(check_days_left(reg))
    df['days_left'] = list_days_left
    return df

In [ ]:
# Function to send email to the vehicle owners.

def send_mail(ownerEmail, reg, days_left):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()

    server.login('youremail@gmail.com', 'password') # Replace email with actual email here. Create an app password with the Google account to use here.
    msg = f'Subject: Your MOT is expiring soon!\n\n'\
        f'Dear Customer,\n\n'\
        f'The MOT for your vehicle with the registration: {reg} expires in {days_left} days!\n\n'\
        f'Call us now at 0123456789 to book your MOT appointment or visit us at ABC Autos Ltd, 1 London Road, London.'\
        f'\n\n'\
        f'Kind regards,\n\nThe ABC team'

    server.sendmail(
        'youremail@gmail.com', # Replace email with actual email.
        ownerEmail,
        msg
    )
    print('Email sent!')


    server.quit()

In [ ]:
account_sid = '...' # Twilio account sid
auth_token = '...' # Twilio auth_token

In [ ]:
# Function to send sms to vehicle owners.

def send_twilio(ownerNumber, reg, days_left):
    client = Client(account_sid, auth_token)
    client.messages.create(
        to=ownerNumber,
        from_='twilio phone number goes here', # replace with the twilio phone number
        body=f"Dear Customer,\n\n"
    f"The MOT for your vehicle: {reg} expires in {days_left} days!\n\n"
    f"Call us now at 0123456789 to book your MOT appointment.\n\n"
    f"Kind regards,\n\nThe ABC team"

    )
    print('Message sent!')

In [ ]:
# Function to call the send_email() and send_twilio() functions.
# Added counters to keep track of how many emails/texts sent.

def send_reminder():
    mailCounter = 0
    textCounter = 0
    for row in df.iterrows():
        print(row[1][3])
        if row[1][3] < 30: # if there are less than 30 days left for any vehicle's MOT, it will send them an email and text.
            send_mail(ownerEmail=row[1][1], reg=row[1][0], days_left=row[1][3])
            mailCounter+=1
            send_twilio(ownerNumber=row[1][2], reg=row[1][0], days_left=row[1][3])
            textCounter+=1
        print(f'Email counter = {mailCounter}')
        print(f'Text counter = {textCounter}')

In [ ]:
# Function to call get_days_left() and send_reminder() which should loop every 24 hours.
# Every 24 hours, the program will pull new data from the MOT database.
# 
while True:
    get_days_left()
    df['mobile'] = df['mobile'].astype(str)
    send_reminder()
    time.sleep(86400)